In [1]:
import pandas as pd
import numpy as np
from utils import reduce_mem_usage

In [2]:
# Constant definitions

def load(filename: str) -> pd.DataFrame:
    return pd.read_csv(f"{DATADIR}/{filename}")

DATADIR       = "Dataset"
SALES_FILE    = "sales_train_validation.csv"
PRICES_FILE   = "sell_prices.csv"
CALENDAR_FILE = "calendar.csv"

ID_VARS        = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]

In [3]:
# Load data.

sales_df    = load(SALES_FILE)
prices_df   = load(PRICES_FILE)
calendar_df = load(CALENDAR_FILE)

In [4]:
# Add ID column to calendar df.

calendar_df["calendar_id"]  = "d_" + (calendar_df.index + 1).astype(str)

# Baseline: average of last `n` days.

In [5]:
n = 28

last_n_days = list(sales_df.columns[-n:])
last_df = sales_df[ID_VARS + last_n_days].copy()
last_df = last_df.melt(id_vars=ID_VARS, value_name="target", var_name="d").reset_index(drop=True)
last_df = last_df.merge(calendar_df, how="inner", left_on="d", right_on="calendar_id").drop("calendar_id", axis=1)

In [10]:
# Our prediction is the average of the last `n` days grouped by weekday.

by_weekday_df = last_df.groupby(ID_VARS + ['wday'], as_index=False).agg({"target": "mean"})

In [11]:
# Load test data.
# Only columns used for scoring and merging are kept.

test_cols = ID_VARS + ["d", "wday", "target"]
test_df = (
    load("sales_test_validation.csv")
    .melt(id_vars=ID_VARS, value_name="target", var_name="d").reset_index(drop=True)
    .merge(calendar_df, how="inner", left_on="d", right_on="calendar_id")
    .drop("calendar_id", axis=1)
    .loc[:, test_cols]
)

In [12]:
# Merge `by_weekday_df` with test data.

on = ID_VARS + ["wday"]
sort_by = ID_VARS + ["d"]
results_melt_df = test_df.merge(by_weekday_df, how="inner", on=on, suffixes=("_true", "_pred")).sort_values(sort_by).reset_index(drop=True).drop("wday", axis=1)
results_melt_df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,target_true,target_pred
0,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1914,2,3.00
1,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1915,0,0.75
2,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1916,0,1.50
3,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1917,0,0.25
4,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1918,0,1.50


In [20]:
# Result aggregated by item id.
# Equivalently, we could aggregate the training data and compute the average on the sum.

results_melt_df.groupby(["item_id", "d"], as_index=False).agg({"target_true": "sum", "target_pred": "sum"}).head(10)

,item_id,d,target_true,target_pred
0,FOODS_1_001,d_1914,4,6.50
1,FOODS_1_001,d_1915,5,2.75
2,FOODS_1_001,d_1916,7,5.50
3,FOODS_1_001,d_1917,1,3.75
4,FOODS_1_001,d_1918,9,4.00
5,FOODS_1_001,d_1919,3,6.50
6,FOODS_1_001,d_1920,9,11.25
7,FOODS_1_001,d_1921,1,6.50
8,FOODS_1_001,d_1922,10,2.75
9,FOODS_1_001,d_1923,3,5.50
